# Predicting Option Price using American Binomial Options

In this case ABC company uses binomial American call option pricing to determine the fair value of Company XYZ's participation rights in the X project. We note that:
ABC assumes that the stock price of X can either up or down and has a terminal value. However, based on information from XYZ management, the X project has a duration of only 20 years, making it unlikely for the project to have a terminal value. Additionally, the stock price of the X project is likely to decline over time, as future cash flows obtained by XYZ decrease.

## Import Libraries

In [1]:
import pandas as pd
import numpy as np

## Data Inputs

Due to the XYZ management, the participating right is set to 5 years. During this duration ABC has the option to buy the underlying asset.

In [2]:
# initial parameters

period = 5 # in years
numberofDays = 252 * period #252 trading days each year

The parameters to determine the budgeted capex cost of Project X are adjusted every 6 months during the participation right. Therefore, we need to create a 6-month array before assigning the parameters into the daily array.

In [3]:
# initiate 6 month array

basePrice_6month          = [29000,    29000,    29000,    29000,     29000,     29000,      29000,      29000,      29000,      29000,      29000  ]
interest_6month           = [0.00,     0.04,     0.08,     0.12,      0.16,      0.20,       0.24,       0.28,       0.32,       0.36,       0.40   ]
fixAdjustment_6month      = [1.0203,   1.0203,   1.0217,   1.0217,    1.0217,    1.0217,     1.0217,     1.0217,     1.0217,     1.0217,     1.0217 ]
incentives_6month         = [0,        0,        0.05,     0.05,      0.10,      0.10,       0.10,       0.10,       0.05,       0.05,       0.05   ]
nameplateCapacity_6month  = [120000,   120000,   120000,   120000,    120000,    120000,     120000,     120000,     120000,     120000,     120000 ]

In [4]:
# initiate daily array to calculate budgeted capex

basePrice = np.zeros(numberofDays+1)
interest = np.zeros(numberofDays+1)
fxAdjustment = np.zeros(numberofDays+1)
incentives = np.zeros(numberofDays+1)
nameplateCapacity = np.zeros(numberofDays+1)

To assign 6-month parameters to the daily array, we need a function that assigns the parameters for each 6-month period.

In [5]:
# assign 6 month array into daily array

def sixmonthToDaily(dailyArray,sixmonthArray):
  j = 0
  for i in range(len(dailyArray)):
    if i>0 and (i % (6 * 21) == 0):
      j = j + 1
    dailyArray[i] = sixmonthArray[j]
  return dailyArray

In [6]:
# convert array from 6 month to daily with functions above

basePrice = sixmonthToDaily(basePrice,basePrice_6month)
interest = sixmonthToDaily(interest,interest_6month)
fxAdjustment = sixmonthToDaily(fxAdjustment,fixAdjustment_6month)
incentives = sixmonthToDaily(incentives,incentives_6month)
nameplateCapacity = sixmonthToDaily(nameplateCapacity,nameplateCapacity_6month)

The budgeted capex cost is determined based on following formula (definitive agreement):

In [7]:
budgetedCapex = basePrice * ((np.ones(numberofDays+1) + interest) * fxAdjustment + incentives) * nameplateCapacity # based on definitive agreement

In [8]:
budgetedCapex = budgetedCapex / 1e6 # convert to million usd

In [9]:
print(f"Budgeted Capex in Million USD: {budgetedCapex}")

Budgeted Capex in Million USD: [3550.644   3550.644   3550.644   ... 5009.50176 5009.50176 5151.7224 ]


In [10]:
# initiate daily array to calculate strike price

capexLoan = np.zeros(numberofDays+1)
accruedCapex = np.zeros(numberofDays+1)
firstprDeduction = np.zeros(numberofDays+1)

In [11]:
# in million usd

capexLoan_6month         = [0,        2096,      1796,      1497,       1197,       898,      599,       299,        0,       0,       0      ]
accruedCapex_6month      = [10,       10,        10,        10,         10,         10,       10,        10,         10,      10,      10     ]
firstprDeduction_6month  = [12.26,    12.26,     12.26,     12.26,      12.26,      12.26,    12.26,     12.26,      12.26,   12.26,   12.26  ]

In [12]:
# convert to daily array

capexLoan = sixmonthToDaily(capexLoan,capexLoan_6month)
accruedCapex = sixmonthToDaily(accruedCapex,accruedCapex_6month)
firstprDeduction = sixmonthToDaily(firstprDeduction,firstprDeduction_6month)

The exercise is determined using the following formula (definitive agreement):

In [13]:
# calculate exercise price

exercisePrice = budgetedCapex - capexLoan - accruedCapex - firstprDeduction
exercisePrice = 0.3 * exercisePrice # 30% of underlying asset will be traded based on definitive agreement

In [14]:
print(f"Exercise Price in Million USD: {exercisePrice}")

Exercise Price in Million USD: [1058.5152   1058.5152   1058.5152   ... 1496.172528 1496.172528
 1538.83872 ]


## Calculate Binomial

The following parameters are used to determine up move, down move, and their probabilities of the underlying asset's price.

In [15]:
# initiates american binomial option parameters

volatility = 0.5282
rf = 0.0438 # risk free rate
dt = period / numberofDays # delta time
underlyingAsset0 = 3683.1 * 0.3 # 30% of underlying asset will be traded based on definitive agreement

In [16]:
# calculate up down move

up = np.exp(volatility * np.sqrt(dt))
down = 1 / up
pu = (np.exp(rf * dt) - down) / (up - down) # probability up
pd = 1 - pu # probability down

In [17]:
# initiate matrix

optionPrice = np.zeros((numberofDays+1, numberofDays+1))
underlyingAsset = np.zeros((numberofDays+1, numberofDays+1))

After that, we calculate the underlying asset movement in daily basis during participating right duration.

In [18]:
# calculate underlying asset movement

for j in range(numberofDays + 1):
  underlyingAsset[j, numberofDays] = underlyingAsset0 * (up ** (numberofDays - j)) * (down ** j)

for i in range(numberofDays - 1, -1, -1):
  for j in range(i + 1):
    underlyingAsset[j, i] = underlyingAsset[j, i + 1] / up

underlyingAsset

array([[1.10493000e+03, 1.14231335e+03, 1.18096149e+03, ...,
        1.66742187e+21, 1.72383613e+21, 1.78215906e+21],
       [0.00000000e+00, 1.06877006e+03, 1.10493000e+03, ...,
        1.56007157e+21, 1.61285382e+21, 1.66742187e+21],
       [0.00000000e+00, 0.00000000e+00, 1.03379349e+03, ...,
        1.45963259e+21, 1.50901667e+21, 1.56007157e+21],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        7.32190413e-16, 7.56962776e-16, 7.82573269e-16],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 7.08228750e-16, 7.32190413e-16],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 6.85051256e-16]])

Finally, after calculating the underlying asset movement, we calculate the option price of executing (buying) X project during participating right.

In [19]:
# calculate option price

for j in range(numberofDays + 1):
  optionPrice[j, numberofDays] = max(0, underlyingAsset[j, numberofDays] - exercisePrice[j])

for i in range(numberofDays - 1, -1, -1):
  for j in range(i + 1):
    optionPrice[j, i] = max(0, underlyingAsset[j, i] - exercisePrice[j], np.exp(-rf * dt) * (pu * optionPrice[j, i + 1] + (1 - pu) * optionPrice[j + 1, i + 1]))


optionPrice

array([[7.10014608e+02, 7.44858745e+02, 7.81083105e+02, ...,
        1.66742187e+21, 1.72383613e+21, 1.78215906e+21],
       [0.00000000e+00, 6.76200834e+02, 7.09707872e+02, ...,
        1.56007157e+21, 1.61285382e+21, 1.66742187e+21],
       [0.00000000e+00, 0.00000000e+00, 6.43682363e+02, ...,
        1.45963259e+21, 1.50901667e+21, 1.56007157e+21],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

So, from above calculation, the option price to buy 30% of underlying asset of X project:

In [20]:
print(f"Option Price: {optionPrice[0][0]} million USD")

Option Price: 710.0146079070705 million USD
